In [ ]:
import prop_simulate
import prop_design
import optimization_NSGA2
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import pandas as pd

In [ ]:
VelocityVector_m_s = [0, 3, 6, 9, 12, 15, 18]
MotorKV_rpm_V = 450
MotorNoLoadCurrent_A = 1.4
MaxCurrent_A = 54
MotorResistance_ohm = 0.032
BatteryVoltage_V = 22.2 #DEIXAR VOLTAGEM LIVRE -> SE PASSAR DE 22.2 DESCARTA
Power_W = 700

CurrentOutput_A = Power_W/BatteryVoltage_V

Convergence = 0.001 

SpinnerCutoff = 0.2
NumberOfStations = 11
NumberOfBlades = 2

In [ ]:
def Evaluation(Chromossome):
    Diameter_m = Chromossome["Diameter"]
    Croot_adim = Chromossome["Croot"]
    Cmax_adim = Chromossome["Cmax"]
    Smax_adim = Chromossome["Smax"]
    Ctip_adim = Chromossome["Ctip"]
    Pitch_m = Chromossome["Pitch"]
    Colective_deg = Chromossome["Collective"]

    #c_R = (a*r**3 + b*r**2 + c*r + d)**0.5
    a = -(2*Cmax_adim**2*Smax_adim - Cmax_adim**2 + Croot_adim**2*Smax_adim**2 - 2*Croot_adim**2*Smax_adim + Croot_adim**2 - Ctip_adim**2*Smax_adim**2)/(Smax_adim**2*(Smax_adim - 1)**2)
    b = -(- 3*Cmax_adim**2*Smax_adim**2 + Cmax_adim**2 - 2*Croot_adim**2*Smax_adim**3 + 3*Croot_adim**2*Smax_adim**2 - Croot_adim**2 + 2*Ctip_adim**2*Smax_adim**3)/(Smax_adim**2*(Smax_adim - 1)**2)
    c = -(3*Cmax_adim**2*Smax_adim - 2*Cmax_adim**2 + Croot_adim**2*Smax_adim**3 - 3*Croot_adim**2*Smax_adim + 2*Croot_adim**2 - Ctip_adim**2*Smax_adim**3)/(Smax_adim*(Smax_adim - 1)**2)
    d = Croot_adim**2

    S = np.linspace(0, 0.9999, NumberOfStations, endpoint = True)
    RadialStations_adim = [SpinnerCutoff + ((1 - SpinnerCutoff)*s) for s in S]
    RadialStations_m = [r*Diameter_m/2 for r in RadialStations_adim]

    ChordDistribution_m = [((a*s**3 + b*s**2 + c*s + d)**0.5)*Diameter_m/2 for s in S]

    TwistDistribution_deg = prop_design.simple_pitch(RadialStations_m, Pitch_m)
    TwistDistributionCollective_deg = [twist + Colective_deg for twist in TwistDistribution_deg]

    for AxialVelocity_m_s in [0, 3, 6, 9, 12, 15]:
        UpperOmega_rad_s = 1000
        LowerOmega_rad_s = 1

        dT_vector, dQ_vector, r_vector, Re_vector, WA_vector, Cl_vector, Cd_vector = prop_simulate.qprop_PrePolarsPreDataframe(AxialVelocity_m_s, UpperOmega_rad_s, NumberOfBlades, Diameter_m/2, RadialStations_m, TwistDistributionCollective_deg, ChordDistribution_m, CLPolars = ClPolars, CDPolars = CdPolars)
        UpperResidue = ObjectiveThrust_N - np.trapz(dT_vector, r_vector)
        dT_vector, dQ_vector, r_vector, Re_vector, WA_vector, Cl_vector, Cd_vector = prop_simulate.qprop_PrePolarsPreDataframe(AxialVelocity_m_s, LowerOmega_rad_s, NumberOfBlades, Diameter_m/2, RadialStations_m, TwistDistributionCollective_deg, ChordDistribution_m, CLPolars = ClPolars, CDPolars = CdPolars)
        LowerResidue = ObjectiveThrust_N - np.trapz(dT_vector, r_vector)

        Converged = False
        for _ in range(500):

            MidOmega_rad_s = (UpperOmega_rad_s + LowerOmega_rad_s)/2
            dT_vector, dQ_vector, r_vector, Re_vector, WA_vector, Cl_vector, Cd_vector = prop_simulate.qprop_PrePolarsPreDataframe(AxialVelocity_m_s, MidOmega_rad_s, NumberOfBlades, Diameter_m/2, RadialStations_m, TwistDistributionCollective_deg, ChordDistribution_m, CLPolars = ClPolars, CDPolars = CdPolars)
            MidResidue = ObjectiveThrust_N - np.trapz(dT_vector, r_vector)

            if abs(MidResidue) < Convergence:
                Converged = True
                # print("===================================================================================================")
                break

            if UpperResidue*MidResidue < 0:
                LowerResidue = MidResidue
                LowerOmega_rad_s = MidOmega_rad_s
            elif LowerResidue*MidResidue < 0:
                UpperResidue = MidResidue
                UpperOmega_rad_s = MidOmega_rad_s
            else:
                print("Convergence Failed")
                break
            
        if not Converged:
            return [0,0]

        return [1,1]
        # Omega_rad_s = 1
        # PropellerTorque_Nm = 0
        # #MotorVoltage(Omega_rad_s, TorquePropeller_Nm)
        # for _ in range(50):
        #     #calcular torque requerido pela hélice no OMG atual
        #     #MotorTorque(Omega_rad_s, Voltage)
        #     #Resíduos
        #     #Se voltagem for maior que a da bateria, fazer algo
        #     #Iterar
        #     pass


    #CurrentOutput_A = (BatteryVoltage_V - (RPM/KV_rpm_V))/MotorResistance_ohm
    def MotorTorque(Omega_rad_s, Voltage_V):
        #inputs: Omega, Voltagem, Parametros do motor
        #outputs: Torque, Torque_OMG, Torque_VOLT, Current, Current_OMG, Current_VOLT
        MotorKV_rad_sV = MotorKV_rpm_V*2*np.pi/60
        MotorVoltage_V = Omega_rad_s/MotorKV_rad_sV
        dMotorVoltage_V__dOmega_rad_s = 1/MotorKV_rad_sV

        MotorCurrent_A = (Voltage_V - MotorVoltage_V)/MotorResistance_ohm
        dMotorCurrent_A__dOmega_rad_s = - dMotorVoltage_V__dOmega_rad_s/MotorResistance_ohm
        dMotorCurrent_A__dVoltage_V = 1/MotorResistance_ohm

        MotorTorque_Nm = (MotorCurrent_A - MotorNoLoadCurrent_A)/MotorKV_rad_sV
        dMotorTorque_Nm__dOmega_rad_s = dMotorCurrent_A__dOmega_rad_s/MotorKV_rad_sV
        dMotorTorque_Nm__dVoltage_V = dMotorCurrent_A__dVoltage_V/MotorKV_rad_sV
        
        return MotorTorque_Nm, dMotorTorque_Nm__dOmega_rad_s, dMotorTorque_Nm__dVoltage_V, MotorCurrent_A, dMotorCurrent_A__dOmega_rad_s, dMotorCurrent_A__dVoltage_V
    def MotorVoltage(Omega_rad_s, PropellerTorque_Nm):
        #inputs: Omega, Torque, Parametros do motor
        #outputs: Volt, Volt_OMG, Volt_TORQUE, Amp, Amp_OMG, Amp_TORQUE
        
        MotorKV_rad_sV = MotorKV_rpm_V*2*np.pi/60
        MotorCurrent_A = (PropellerTorque_Nm*MotorKV_rad_sV) + MotorNoLoadCurrent_A
        Voltage_V = (MotorCurrent_A*MotorResistance_ohm) + (Omega_rad_s/MotorKV_rad_sV)

        for _ in range(50):
            MotorTorque_Nm, dMotorTorque_Nm__dOmega_rad_s, dMotorTorque_Nm__dVoltage_V, MotorCurrent_A, dMotorCurrent_A__dOmega_rad_s, dMotorCurrent_A__dVoltage_V = MotorTorque(Omega_rad_s, Voltage_V)
            Residue1 = MotorTorque_Nm - PropellerTorque_Nm
            Residue1_VOLT = dMotorTorque_Nm__dVoltage_V
            dVoltage_V = -Residue1/Residue1_VOLT
            

def Validation(Chromossome):
    Diameter_m = Chromossome["Diameter"]
    Croot_adim = Chromossome["Croot"]
    Cmax_adim = Chromossome["Cmax"]
    Smax_adim = Chromossome["Smax"]
    Ctip_adim = Chromossome["Ctip"]
    Pitch_m = Chromossome["Pitch"]
    Colective_deg = Chromossome["Collective"]

    #c_R = (a*r**3 + b*r**2 + c*r + d)**0.5
    a = -(2*Cmax_adim**2*Smax_adim - Cmax_adim**2 + Croot_adim**2*Smax_adim**2 - 2*Croot_adim**2*Smax_adim + Croot_adim**2 - Ctip_adim**2*Smax_adim**2)/(Smax_adim**2*(Smax_adim - 1)**2)
    b = -(- 3*Cmax_adim**2*Smax_adim**2 + Cmax_adim**2 - 2*Croot_adim**2*Smax_adim**3 + 3*Croot_adim**2*Smax_adim**2 - Croot_adim**2 + 2*Ctip_adim**2*Smax_adim**3)/(Smax_adim**2*(Smax_adim - 1)**2)
    c = -(3*Cmax_adim**2*Smax_adim - 2*Cmax_adim**2 + Croot_adim**2*Smax_adim**3 - 3*Croot_adim**2*Smax_adim + 2*Croot_adim**2 - Ctip_adim**2*Smax_adim**3)/(Smax_adim*(Smax_adim - 1)**2)
    d = Croot_adim**2

    DerivativeZero = c/(2*(d**0.5))
    if DerivativeZero < 0:
        return False
    DerivativeOne = (3*a + 2*b + c)/(2*((a + b + c + d)**0.5))
    if DerivativeOne > 0:
        return False
    return True